In [1]:
import polars as pl
import numpy as np

from pickle import dump
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from scipy.spatial.transform import Rotation as R

import random
import os
import time

In [2]:
INPUT_DIR = "../../input/cmi-detect-behavior-with-sensor-data/"
OUTPUT_DIR = "../../models/debug/"

In [3]:
def handle_quaternion_missing_values(rot_data: np.ndarray) -> np.ndarray:
    """
    Handle missing values in quaternion data intelligently
    
    Key insight: Quaternions must have unit length |q| = 1
    If one component is missing, we can reconstruct it from the others
    """
    rot_cleaned = rot_data.copy()
    
    for i in range(len(rot_data)):
        row = rot_data[i]
        missing_count = np.isnan(row).sum()
        
        if missing_count == 0:
            # No missing values, normalize to unit quaternion
            norm = np.linalg.norm(row)
            if norm > 1e-8:
                rot_cleaned[i] = row / norm
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]  # Identity quaternion
                
        elif missing_count == 1:
            # One missing value, reconstruct using unit quaternion constraint
            # |w|² + |x|² + |y|² + |z|² = 1
            missing_idx = np.where(np.isnan(row))[0][0]
            valid_values = row[~np.isnan(row)]
            
            sum_squares = np.sum(valid_values**2)
            if sum_squares <= 1.0:
                missing_value = np.sqrt(max(0, 1.0 - sum_squares))
                # Choose sign for continuity with previous quaternion
                if i > 0 and not np.isnan(rot_cleaned[i-1, missing_idx]):
                    if rot_cleaned[i-1, missing_idx] < 0:
                        missing_value = -missing_value
                rot_cleaned[i, missing_idx] = missing_value
                rot_cleaned[i, ~np.isnan(row)] = valid_values
            else:
                rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
        else:
            # More than one missing value, use identity quaternion
            rot_cleaned[i] = [1.0, 0.0, 0.0, 0.0]
    
    return rot_cleaned

# thanks: https://www.kaggle.com/code/nksusth/lb-0-78-quaternions-tf-bilstm-gru-attention
def calculate_angular_velocity_from_quat(rot_data, time_delta=1/10): # Assuming 10Hz sampling rate
    if isinstance(rot_data, pl.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].to_numpy()
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_vel = np.zeros((num_samples, 3))

    for i in range(num_samples - 1):
        q_t = quat_values[i]
        q_t_plus_dt = quat_values[i+1]

        if np.all(np.isnan(q_t)) or np.all(np.isclose(q_t, 0)) or \
           np.all(np.isnan(q_t_plus_dt)) or np.all(np.isclose(q_t_plus_dt, 0)):
            continue

        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)

            # Calculate the relative rotation
            delta_rot = rot_t.inv() * rot_t_plus_dt
            
            # Convert delta rotation to angular velocity vector
            # The rotation vector (Euler axis * angle) scaled by 1/dt
            # is a good approximation for small delta_rot
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError:
            # If quaternion is invalid, angular velocity remains zero
            pass
            
    return angular_vel

# thanks: https://www.kaggle.com/code/nksusth/lb-0-78-quaternions-tf-bilstm-gru-attention
def calculate_angular_distance(rot_data):
    if isinstance(rot_data, pl.DataFrame):
        quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].to_numpy()
    else:
        quat_values = rot_data

    num_samples = quat_values.shape[0]
    angular_dist = np.zeros(num_samples)

    for i in range(num_samples - 1):
        q1 = quat_values[i]
        q2 = quat_values[i+1]

        if np.all(np.isnan(q1)) or np.all(np.isclose(q1, 0)) or \
           np.all(np.isnan(q2)) or np.all(np.isclose(q2, 0)):
            angular_dist[i] = 0 # Или np.nan, в зависимости от желаемого поведения
            continue
        try:
            # Преобразование кватернионов в объекты Rotation
            r1 = R.from_quat(q1)
            r2 = R.from_quat(q2)

            # Вычисление углового расстояния: 2 * arccos(|real(p * q*)|)
            # где p* - сопряженный кватернион q
            # В scipy.spatial.transform.Rotation, r1.inv() * r2 дает относительное вращение.
            # Угол этого относительного вращения - это и есть угловое расстояние.
            relative_rotation = r1.inv() * r2
            
            # Угол rotation vector соответствует угловому расстоянию
            # Норма rotation vector - это угол в радианах
            angle = np.linalg.norm(relative_rotation.as_rotvec())
            angular_dist[i] = angle
        except ValueError:
            angular_dist[i] = 0 # В случае недействительных кватернионов
            pass
            
    return angular_dist

def preprocess_left_handed(l_tr):
    rot_cleaned = handle_quaternion_missing_values(l_tr[["rot_w","rot_x", "rot_y", "rot_z"]].to_numpy())
    rot_scipy = rot_cleaned[:, [1, 2, 3, 0]]
    
    norms = np.linalg.norm(rot_scipy, axis=1)
    if np.any(norms < 1e-8):
        # Replace problematic quaternions with identity
        mask = norms < 1e-8
        rot_scipy[mask] = [0.0, 0.0, 0.0, 1.0]  # Identity quaternion in scipy format
        print("yes")
    
    r = R.from_quat(rot_scipy)
    tmp = r.as_euler("xyz")
    tmp[:,1] = - tmp[:,1]
    tmp[:,2] = - tmp[:,2]
    r = R.from_euler("xyz", tmp)
    tmp = r.as_quat()
    l_tr = l_tr.with_columns(pl.DataFrame(tmp, schema=["rot_x", "rot_y", "rot_z", "rot_w"]))
    l_tr = l_tr.with_columns(-pl.col("acc_x"))
    
    tmp = l_tr[["thm_3", "thm_5"]]
    tmp.columns = ["thm_5", "thm_3"]
    l_tr = l_tr.with_columns(tmp)
    
    swap_1_2_4_base = [[0,7],[1,6],[2,5],[3,4], [4,3], [5,2],[6,1],[7,0]]
    swap_3_5_base = [[0,56],[8,48],[16,40], [24,32],[32,24], [40,16],[48,8], [56,0]]
    
    swap_1_2_4 = list()
    for i in range(0,64,8):
        ll = list()
        for (k,l) in swap_1_2_4_base:
            ll.append([k+i, l+i])
        swap_1_2_4 += ll
    
    swap_3_5 = list()
    for i in range(8):
        ll = list()
        for (k,l) in swap_3_5_base:
            ll.append([k+i, l+i])
        swap_3_5 += ll
    
    l_df = l_tr
    
    for (k,l) in zip(["tof_3_v" + str(x) for x in range(64)], ["tof_5_v" + str(x) for x in range(64)]):
        l_tr = l_tr.with_columns(l_df[k].alias(l))
    
    for (k,l) in zip(["tof_3_v" + str(x) for x in range(64)], ["tof_5_v" + str(x) for x in range(64)]):
        l_tr = l_tr.with_columns(l_df[l].alias(k))
    
    l_df = l_tr
    
    for i in [1,2,4]:
        for (k, l) in swap_1_2_4:
            l_tr = l_tr.with_columns(l_df["tof_" + str(i) + "_v"+str(k)].alias("tof_" + str(i) + "_v"+str(l)))
    
    for i in [3,5]:
        for (k, l) in swap_3_5:
            l_tr = l_tr.with_columns(l_df["tof_" + str(i) + "_v"+str(k)].alias("tof_" + str(i) + "_v"+str(l)))
    return l_tr

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda', index=0)

In [6]:
tr = pl.read_csv(os.path.join(INPUT_DIR,"train.csv"))
no_gesture = 'SEQ_011975'
tr = tr.filter(pl.col("sequence_id") != no_gesture)

#null_seqs = pl.read_csv("../input/null_seqs.csv")
#tr = tr.filter(~pl.col("sequence_id").is_in(null_seqs[:,0].implode()))

demo = pl.read_csv(os.path.join(INPUT_DIR,"train_demographics.csv"))

In [7]:
r_subjs = demo.filter(pl.col("handedness") == 1)["subject"].to_list()
l_subjs = demo.filter(pl.col("handedness") == 0)["subject"].to_list()

r_subjs.remove("SUBJ_019262") # these subjects wore the device on the wrong side of the wrist and score worse then left handers.
r_subjs.remove("SUBJ_045235") # further, tof and thm are probably not salvagable, but maybe imu can be adjusted, could look into later

r_new = tr.filter(pl.col("subject").is_in(r_subjs))
l_new = tr.filter(pl.col("subject").is_in(l_subjs))

l_new = preprocess_left_handed(l_new)
 
tr = pl.concat([r_new, l_new])
tr = tr.sort(by="row_id")

In [8]:
labels = tr.filter(pl.col("sequence_counter") ==0)[:,["sequence_id", "subject","gesture"]]
labels = labels.sort(by="gesture")
label_encoder = LabelEncoder()
labels = labels.with_columns(pl.Series(label_encoder.fit_transform(labels["gesture"])).alias("gesture"))
oh_encoder = OneHotEncoder()
onehotted = oh_encoder.fit_transform(pl.DataFrame(labels["gesture"])).toarray()
onehotted = pl.DataFrame(onehotted, orient="row", schema=label_encoder.classes_.tolist())
labels = labels.hstack(onehotted)
labels_df = labels.sort(by="sequence_id")

In [9]:
def tof_to_array(df, sensor, t):
    l = list()
    for i in range(8):
        ll = list()
        for j in range(8):
            ll.append(df[t,f"tof_{sensor}_v{i*8+j}"])
        l.append(ll)
    return np.array(l)

def tofdiff_to_array(df, sensor, t):
    l = list()
    for i in range(8):
        ll = list()
        for j in range(8):
            ll.append(df[t,f"tof_{sensor}_v{i*8+j}_diff"])
        l.append(ll)
    return np.array(l)

In [10]:
n_seqs = tr.filter(pl.col("sequence_counter") == 0).shape[0]

In [11]:
%%time
tof_arr = np.zeros((n_seqs, 5, 128, 8, 8))
for e, (idx, df) in enumerate(tr.group_by("sequence_id", maintain_order=True)):
    start = max(df.shape[0]-128, 0)
    offset = max(128-df.shape[0], 0)
    for ee, i in enumerate(range(start, df.shape[0])):
        for j in range(1,6):
            tof_image = tof_to_array(df, j, i)
            tof_arr[e,j-1,offset+ee,:,:] = tof_image

CPU times: user 2min 52s, sys: 13.7 s, total: 3min 6s
Wall time: 2min 33s


In [12]:
tr = tr.with_columns(pl.all().replace(-1, None))

In [13]:
dfs = list()
for e, (idx, df) in enumerate(tr.group_by("sequence_id", maintain_order = True)):
    df = df.with_columns(pl.lit(e).alias("subject_index"))
    for i in [f"tof_{x}_v{y}" for x in range(1,6) for y in range(64)]:
        df = df.with_columns(pl.col(i).diff().alias(i+"_diff"))
    
    dfs.append(df)
tr = pl.concat(dfs)

In [14]:
%%time
tof_diff_arr = np.zeros((n_seqs, 10, 128, 8, 8))
for e, (idx, df) in enumerate(tr.group_by("sequence_id", maintain_order=True)):
    start = max(df.shape[0]-128, 0)
    offset = max(128-df.shape[0], 0)
    for ee, i in enumerate(range(start, df.shape[0])):
        for j in range(1,6):
            tof_image = tof_to_array(df, j, i)
            tof_diff_arr[e,j-1,offset+ee,:,:] = tof_image
        for j in range(1,6):
            tof_image = tofdiff_to_array(df, j, i)
            tof_diff_arr[e,5+j-1,offset+ee,:,:] = tof_image

CPU times: user 5min 56s, sys: 12.8 s, total: 6min 9s
Wall time: 5min 17s


In [15]:
def standard_scale_3d_fit_transform(data):
    mean = np.nanmean(data, axis=(0,2))
    
    std = np.nanstd(data, axis=(0,2))
    
    mean = mean[np.newaxis,:,np.newaxis,:,:]
    std = std[np.newaxis,:,np.newaxis,:,:]
    
    scaled_data = (data - mean) / std
    return scaled_data, mean, std

def standard_scale_3d_transform(data, mean, std):
    return (data - mean) / std


def prep_data_tof3d(tof_arr, labels_df):
    tr_tof, mean, std = standard_scale_3d_fit_transform(tof_arr)
    tr_tof = np.nan_to_num(tr_tof, 0)
    y_tr = labels_df[:,3:].to_numpy()

    return tr_tof, y_tr, mean, std

In [16]:
class TestTimeDataset3d(torch.utils.data.Dataset):
    def __init__(self, tof_arr, targets):
        self.targets = targets
        self.tof_arr = tof_arr.astype(np.float32)
        
    def __getitem__(self, index):
        y = self.targets[index]
        tof = self.tof_arr[index]
        
        return tof, y
    
    def __len__(self):
        return len(self.tof_arr)

In [17]:
class ResidualBlock3d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding="same")
        self.bn1 = nn.BatchNorm3d(num_features=out_channels)
        self.conv2 = nn.Conv3d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding="same")
        self.bn2 = nn.BatchNorm3d(num_features=out_channels)
        
        if in_channels != out_channels:
            self.res_layer = torch.nn.Conv3d(in_channels, out_channels, kernel_size=1, padding="same")
        else:
            self.res_layer = None
            
    def forward(self, x):
        res = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        if self.res_layer is None:
            x = torch.add(x, res)
        else: 
            x = torch.add(x, self.res_layer(res)) # should the res layer have a relu? or batch norm?
            # TODO experiment with the residual block
        return x 


class RNNet3d(nn.Module):
    def __init__(self, input_chan, chan=128):
        super().__init__()
        self.block0 = ResidualBlock3d(input_chan, chan, 3)
        self.block1 = ResidualBlock3d(chan, chan, 3)
        self.block2 = ResidualBlock3d(chan, chan, 3)

        self.rnn = nn.GRU(chan, 128,
            num_layers = 2, 
            batch_first =True,
            bidirectional=True)
        
        self.pool = nn.MaxPool3d((1,2,2))
        self.fc0 = nn.Linear(256, 256)
        self.drop = nn.Dropout(0.2)
        self.fc1 = nn.Linear(256, 18)
            
        
    def forward(self, x):
        x = self.block0(x)
        x = self.pool(x)
        x = self.block1(x)
        x = self.pool(x)
        x = self.block2(x)
        x = self.pool(x)
        
        x = x.squeeze(dim=[3,4])
        x = x.permute(0, 2, 1)
        x,_ = self.rnn(x)
        x = self.drop(F.relu(self.fc0(x)))
        x = self.fc1(x)
        x = x[:,-30:,:].mean(axis=1)
        return x

In [19]:
x_tr_tof, y_tr_tof, tof_mean, tof_std = prep_data_tof3d(tof_arr, labels_df)

np.save(os.path.join(OUTPUT_DIR,"tof_mean.npy"), tof_mean)
np.save(os.path.join(OUTPUT_DIR,"tof_std.npy"), tof_std)

In [20]:
x_tr_tof_diff, y_tr_tof_diff, tof_diff_mean, tof_diff_std = prep_data_tof3d(tof_diff_arr, labels_df)

np.save(os.path.join(OUTPUT_DIR,"tof_diff_mean.npy"), tof_diff_mean)
np.save(os.path.join(OUTPUT_DIR,"tof_diff_std.npy"), tof_diff_std)

In [21]:
start_time = time.time()

In [22]:
seed = 10
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 

In [23]:
for p in range(20):
    net = RNNet3d(input_chan=5).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0005)
    
    tr_dataset = TestTimeDataset3d(x_tr_tof.astype(np.float32), y_tr_tof.astype(np.float32))
    
    trainloader = torch.utils.data.DataLoader(tr_dataset, batch_size=64,
                                              shuffle=True, num_workers=4)

    for epoch in range(60):
        net.train()
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
    model_state = net.state_dict()
    torch.save(model_state, os.path.join(OUTPUT_DIR,f"alldata_tof3dnet_itr{p}"))
    print(p) 
    

0


In [24]:
for p in range(20):

    net = RNNet3d(input_chan=10).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0005)

    
    tr_dataset = TestTimeDataset3d(x_tr_tof_diff.astype(np.float32), y_tr_tof_diff.astype(np.float32))
    
    trainloader = torch.utils.data.DataLoader(tr_dataset, batch_size=64,
                                              shuffle=True, num_workers=4)

    for epoch in range(60):
        total_loss = 0
        for i, data in enumerate(trainloader, 0):

            inputs, labels = data[0].to(device), data[1].to(device)
    
            optimizer.zero_grad()
    
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model_state = net.state_dict()
    torch.save(model_state, os.path.join(OUTPUT_DIR,f"alldata_tof3dnet_diff_itr{p}"))
    print(p) 

0


In [25]:
print((time.time()-start_time)/60/60)

0.005151151948504977
